# Comparing Between Two Experiment 

## Introduction

The aim of this tutorial is to use the experiment set API to compare the results between two experiment using the Experiment and ExperimentSet API. 

In [ ]:
# Load the module 
from IPTK.DataStructure.Experiment import Experiment 
from IPTK.DataStructure.ExperimentalSet import ExperimentSet
from IPTK.IO.InFunctions import load_identification_table
